In [1]:
# imports, including functions provided with the starter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

os.chdir('..')

from groupXY_functions import *

In [4]:
# week11-2_features.pdf, slide 11
def get_compactness(area, perimeter):
    return perimeter**2 / (4 * np.pi * area)

# Return smallest and largest x, smallest and largest y that contains a white pixel, so that we can crop it to measure asymmetry
def get_extremes(image):
    non_zeros = np.nonzero(image)
    return min(non_zeros[1]), max(non_zeros[1]), min(non_zeros[0]), max(non_zeros[0])

# Crops both the image and the mask to the actual size of the mask, saves to files
# Returns cropped image and mask in the array
def crop(image_id):
    path = 'data/example_image'
    path_cropped = path + '_cropped'
    path_mask = 'data/example_segmentation'
    path_mask_cropped = path_mask + '_cropped'
    if os.path.exists(path_cropped + image_id + '.jpg'):
        image = plt.imread(path_cropped + '/' + image_id + '.jpg')
        mask = plt.imread(path_mask_cropped + '/' + image_id + '.png')
        return image, mask
    if not os.path.exists(path_cropped):
        os.mkdir(path_cropped)
        os.mkdir(path_mask_cropped)
    image = plt.imread(path + '/' + image_id + '.jpg')
    mask = plt.imread(path_mask + '/' + image_id + '_segmentation.png')
    min_x, max_x, min_y, max_y = get_extremes(mask)
    mask = mask[min_y:max_y+1, min_x:max_x+1]
    plt.imsave(path_mask_cropped + '/' + image_id + '.png', mask, cmap=plt.cm.gray)
    # TODO: replace irrelevant parts with black
    image = image[min_y:max_y+1, min_x:max_x+1, :]
    plt.imsave(path_cropped + '/' + image_id + '.jpg', image)   
    return mask, image

# Returns asymmetry score of an image in vertical and horizontal axises.
# 0 is a perfect circle, 1 has all the quarters of an image completely different
def asymmetry_score(image):
    print(np.shape(image))
    height, width = np.shape(image)
    if width % 2:
        # halves need to be the same size to compare them
        left_half = image[:, 0:width//2+1]
    else:
        left_half = image[:, 0:width//2]
    right_half = np.flip(image[:, width//2:], 1)
    if height % 2:
        top_half = image[0:height//2+1, :]
    else:
        top_half = image[0:height//2, :]
    bottom_half = np.flip(image[height//2:, :], 0)
    hor_diff = np.where(left_half != right_half, 1, 0)
    ver_diff = np.where(top_half != bottom_half, 1, 0)
    return (np.sum(hor_diff) + np.sum(ver_diff)) / width / height

In [5]:
# Import a data frame with file names and associated diseases
df_conditions = pd.read_csv('data/example_ground_truth.csv')

df_evaluations = pd.DataFrame(df_conditions['image_id'])
for name in ['asymmetry', 'border', 'color']:
    df_evaluations[name] = 0.0
for i in range(df_evaluations.shape[0]):
    image, mask = crop(df_evaluations.at[i, 'image_id'])
    df_evaluations.at[i, 'asymmetry'] = asymmetry_score(mask)
    area, perimeter = measure_area_perimeter(mask)
    df_evaluations.at[i, 'border'] = get_compactness(area, perimeter)

(521, 688, 3)


ValueError: too many values to unpack (expected 2)

In [ ]:
np.sum(df_evaluations["border"] < 1)
df_evaluations

In [ ]:
mask = plt.imread('data/example_segmentation/ISIC_0001769_segmentation.png')
image = plt.imread('data/example_image/ISIC_0001769.jpg')
np.shape(image)